In [1]:
import os
import sys

from datetime import datetime, timedelta

util_dir = os.path.join(os.getcwd(), r'../..', 'util')
stckinfo_dir = os.path.join(os.getcwd(), r'../..', 'util', 'stocks_info')

sys.path.append(util_dir)
sys.path.append(stckinfo_dir)

In [3]:
# Execute kis_kospi_code_mst.py
script_path = r'C:\Users\byeun\workspace\ofij\util\stocks_info\kis_kospi_code_mst.py'
with open(script_path, encoding='utf-8') as f:
    exec(f.read())

Done


In [4]:
# 업종분류코드 다운로드
script_path = r'C:\Users\byeun\workspace\ofij\util\stocks_info\sector_code.py'
with open(script_path, encoding='utf-8') as f:
    exec(f.read())

In [2]:
import kis_auth as ka
import kis_domstk as kb

from kis_kospi_code_mst import kospi_master_download, get_kospi_master_dataframe

ka.auth()

Done


In [19]:
data_dir = os.path.join(os.path.expanduser('~'), 'data','ofij')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

stockinfo_file = os.path.join(data_dir, 'stock_info.feather')
stockprice_file = os.path.join(data_dir, 'stock_prices.feather')
sectorprice_file = os.path.join(data_dir, 'sector_prices.feather')

In [4]:
import time

import pandas as pd
from tqdm import tqdm
from time import sleep

def backoff(retries, delay=1, factor=2):
    """Backoff function to handle retries with exponential delay."""
    def wrapper(func):
        def inner(*args, **kwargs):
            attempt = 0
            while attempt < retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempt += 1
                    if attempt == retries:
                        raise e
                    sleep(delay * (factor ** (attempt - 1)))
        return inner
    return wrapper

In [14]:
dfkospicc = pd.read_csv('kospi_code.csv')

stockcodes = dfkospicc[dfkospicc.그룹코드 == 'ST'].단축코드.tolist()
stock_info_results = {}

@backoff(retries=3, delay=1, factor=2)
def fetch_stock_info(stock_code):
    return kb.get_search_stock_info(pdno=stock_code)

for stock_code in tqdm(stockcodes, desc="Fetching stock info"):
    try:
        stock_info = fetch_stock_info(stock_code)
        stock_info_results[stock_code] = stock_info
        time.sleep(0.1)
    except Exception as e:
        print(f"Failed to fetch stock info for {stock_code}: {e}")


stock_info_df = pd.DataFrame.from_dict(stock_info_results, orient='index')

# save to feather file
stock_info_df.reset_index(inplace=True)
stock_info_df.rename(columns={'index': '단축코드'}, inplace=True)
stock_info_df.to_feather(stockinfo_file)

Fetching stock info:   0%|          | 0/932 [00:00<?, ?it/s]

Fetching stock info: 100%|██████████| 932/932 [14:40<00:00,  1.06it/s]



In [ ]:
@backoff(retries=3, delay=1, factor=2)
def fetch_stock_prices(stock_code, period_code):
    return kb.get_inquire_daily_price(itm_no=stock_code, period_code=period_code)

lst_stock_prices = []

period_codes = ['D', 'W', 'M']
for period_code in period_codes:
    print(f"Fetching stock prices for period {period_code}")
    for stock_code in tqdm(stockcodes, desc=f"Fetching stock prices for period {period_code}"):
        try:
            stock_prices = fetch_stock_prices(stock_code, period_code)
            # add period code to stock_prices dataframe
            stock_prices['PERIOD_CODE'] = period_code
            # add stock_code to stock_prices dataframe if not exist
            if 'STOCK_CODE' not in stock_prices.columns:
                stock_prices['STOCK_CODE'] = stock_code
                
            lst_stock_prices.append(stock_prices)
        except Exception as e:
            print(f"Failed to fetch stock prices for {stock_code} in period {period_code}: {e}")

# concatenate all stock prices dataframes
stock_prices = pd.concat(lst_stock_prices, ignore_index=True)
# save to feather file
stock_prices.reset_index(inplace=True, drop=True)
stock_prices.to_feather(stockprice_file)

Fetching stock prices for period D


Fetching stock prices for period D: 100%|██████████| 932/932 [13:06<00:00,  1.19it/s]


Fetching stock prices for period W


Fetching stock prices for period W: 100%|██████████| 932/932 [13:06<00:00,  1.19it/s]


Fetching stock prices for period M


Fetching stock prices for period M: 100%|██████████| 932/932 [12:58<00:00,  1.20it/s]


#### 업종별 가격 다운로드

In [5]:
sectorcodes = pd.read_csv('idxcode.csv')

In [6]:
today = pd.to_datetime('today')
fr_date = (today - timedelta(days=100)).strftime('%Y%m%d')
to_date = today.strftime('%Y%m%d')

print(f"Fetching sector prices for date range {fr_date} to {to_date}")

Fetching sector prices for date range 20250116 to 20250426


In [ ]:
@backoff(retries=3, delay=1, factor=2)
def fetch_sector_prices(sector_code, period_code, fr_date, to_date):
    return kb.get_domestic_sector_price(iscd=sector_code, date_1=fr_date, date_2=to_date, period_div_code=period_code)

sectorres = []
period_codes = ['D', 'W', 'M']
for period_code in period_codes:
    print(f"Fetching stock prices for period {period_code}")
    for i, rec in tqdm(sectorcodes.iterrows(), desc=f"Fetching sector prices for period {period_code}"):
        sector_code = rec['업종코드']
        sector_name = rec['업종명']
        try:
            res = fetch_sector_prices(sector_code, period_code, fr_date, to_date)
            output1 = res.output1
            output2 = res.output2
            df = pd.DataFrame(output2)
            df['sector_code'] = sector_code
            df['sector_name'] = sector_name
            df['period_code'] = period_code
            sectorres.append(df)
        except Exception as e:
            print(f"Failed to fetch sector prices for {sector_code} in period {period_code}: {e}")

sector_prices = pd.concat(sectorres, ignore_index=True)
# save to feather file
sector_prices.reset_index(inplace=True, drop=True)
sector_prices.to_feather(sectorprice_file)

Fetching stock prices for period D


Fetching sector prices for period D: 0it [00:01, ?it/s]


In [16]:
pd.Series(sectorres[0]['sector_prices'].output1)

bstp_nmix_prdy_vrss      23.97
prdy_vrss_sign               2
bstp_nmix_prdy_ctrt       0.95
prdy_nmix              2522.33
acml_vol                403528
acml_tr_pbmn           8760600
hts_kor_isnm                종합
bstp_nmix_prpr         2546.30
bstp_cls_code             0001
prdy_vol                327935
bstp_nmix_oprc         2544.59
bstp_nmix_hgpr         2553.33
bstp_nmix_lwpr         2534.94
futs_prdy_oprc         2525.34
futs_prdy_hgpr         2530.61
futs_prdy_lwpr         2507.21
dtype: object

In [ ]:
df = pd.DataFrame(sectorres[0]['sector_prices'].output2)
df['sector_code'] = sector_code
df['sector_name'] = sector_name
df['period_code'] = period_code

,stck_bsop_date,bstp_nmix_prpr,bstp_nmix_oprc,bstp_nmix_hgpr,bstp_nmix_lwpr,acml_vol,acml_tr_pbmn,mod_yn
0,20250425,2546.30,2544.59,2553.33,2534.94,403528,8760600,N
1,20250424,2522.33,2525.34,2530.61,2507.21,327935,7272322,N


In [18]:
df

,stck_bsop_date,bstp_nmix_prpr,bstp_nmix_oprc,bstp_nmix_hgpr,bstp_nmix_lwpr,acml_vol,acml_tr_pbmn,mod_yn,sector_code,sector_name,period_code
0,20250425,2546.30,2544.59,2553.33,2534.94,403528,8760600,N,0001,01종합,D
1,20250424,2522.33,2525.34,2530.61,2507.21,327935,7272322,N,0001,01종합,D
2,20250423,2525.56,2520.56,2526.72,2507.10,538116,7764180,N,0001,01종합,D
3,20250422,2486.64,2481.18,2494.59,2476.14,416680,6182471,N,0001,01종합,D
4,20250421,2488.42,2486.38,2498.84,2479.23,428977,5239733,N,0001,01종합,D
5,20250418,2483.42,2473.30,2486.37,2466.90,450985,5649647,N,0001,01종합,D
6,20250417,2470.41,2451.70,2473.24,2447.40,601739,6606299,N,0001,01종합,D
7,20250416,2447.43,2472.78,2475.83,2442.72,613174,6725688,N,0001,01종합,D
8,20250415,2477.41,2464.51,2482.56,2461.23,560340,6510809,N,0001,01종합,D
9,20250414,2455.89,2454.28,2465.56,2449.48,488618,6687794,N,0001,01종합,D


In [18]:
prcdict ={
  "rt_cd": {
    "한글명": "성공 실패 여부",
    "Type": "String",
    "Required": "Y",
    "Length": 1,
    "Description": ""
  },
  "msg_cd": {
    "한글명": "응답코드",
    "Type": "String",
    "Required": "Y",
    "Length": 8,
    "Description": ""
  },
  "msg1": {
    "한글명": "응답메세지",
    "Type": "String",
    "Required": "Y",
    "Length": 80,
    "Description": ""
  },
  "Output": {
    "한글명": "응답상세",
    "Type": "Object",
    "Required": "Y",
    "Description": "",
    "fields": {
      "rprs_mrkt_kor_name": {
        "한글명": "대표 시장 한글 명",
        "Type": "String",
        "Required": "Y",
        "Length": 40,
        "Description": ""
      },
      "new_hgpr_lwpr_cls_code": {
        "한글명": "신 고가 저가 구분 코드",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": "특정 경우에만 데이터 출력"
      },
      "mxpr_llam_cls_code": {
        "한글명": "상하한가 구분 코드",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": "특정 경우에만 데이터 출력"
      },
      "crdt_able_yn": {
        "한글명": "신용 가능 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "stck_mxpr": {
        "한글명": "주식 상한가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "elw_pblc_yn": {
        "한글명": "ELW 발행 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "prdy_clpr_vrss_oprc_rate": {
        "한글명": "전일 종가 대비 시가2 비율",
        "Type": "String",
        "Required": "Y",
        "Length": 84,
        "Description": ""
      },
      "crdt_rate": {
        "한글명": "신용 비율",
        "Type": "String",
        "Required": "Y",
        "Length": 84,
        "Description": ""
      },
      "marg_rate": {
        "한글명": "증거금 비율",
        "Type": "String",
        "Required": "Y",
        "Length": 84,
        "Description": ""
      },
      "lwpr_vrss_prpr": {
        "한글명": "최저가 대비 현재가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "lwpr_vrss_prpr_sign": {
        "한글명": "최저가 대비 현재가 부호",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "prdy_clpr_vrss_lwpr_rate": {
        "한글명": "전일 종가 대비 최저가 비율",
        "Type": "String",
        "Required": "Y",
        "Length": 84,
        "Description": ""
      },
      "stck_lwpr": {
        "한글명": "주식 최저가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "hgpr_vrss_prpr": {
        "한글명": "최고가 대비 현재가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "hgpr_vrss_prpr_sign": {
        "한글명": "최고가 대비 현재가 부호",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "prdy_clpr_vrss_hgpr_rate": {
        "한글명": "전일 종가 대비 최고가 비율",
        "Type": "String",
        "Required": "Y",
        "Length": 84,
        "Description": ""
      },
      "stck_hgpr": {
        "한글명": "주식 최고가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "oprc_vrss_prpr": {
        "한글명": "시가2 대비 현재가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "oprc_vrss_prpr_sign": {
        "한글명": "시가2 대비 현재가 부호",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "mang_issu_yn": {
        "한글명": "관리 종목 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "divi_app_cls_code": {
        "한글명": "동시호가배분처리코드",
        "Type": "String",
        "Required": "Y",
        "Length": 2,
        "Description": "11:매수상한배분 12:매수하한배분 13: 매도상한배분 14:매도하한배분"
      },
      "short_over_yn": {
        "한글명": "단기과열여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "mrkt_warn_cls_code": {
        "한글명": "시장경고코드",
        "Type": "String",
        "Required": "Y",
        "Length": 2,
        "Description": "00: 없음 01: 투자주의 02:투자경고 03:투자위험"
      },
      "invt_caful_yn": {
        "한글명": "투자유의여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "stange_runup_yn": {
        "한글명": "이상급등여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "ssts_hot_yn": {
        "한글명": "공매도과열 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "low_current_yn": {
        "한글명": "저유동성 종목 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "vi_cls_code": {
        "한글명": "VI적용구분코드",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "short_over_cls_code": {
        "한글명": "단기과열구분코드",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "stck_llam": {
        "한글명": "주식 하한가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "new_lstn_cls_name": {
        "한글명": "신규 상장 구분 명",
        "Type": "String",
        "Required": "Y",
        "Length": 40,
        "Description": ""
      },
      "vlnt_deal_cls_name": {
        "한글명": "임의 매매 구분 명",
        "Type": "String",
        "Required": "Y",
        "Length": 16,
        "Description": ""
      },
      "flng_cls_name": {
        "한글명": "락 구분 이름",
        "Type": "String",
        "Required": "Y",
        "Length": 40,
        "Description": "특정 경우에만 데이터 출력"
      },
      "revl_issu_reas_name": {
        "한글명": "재평가 종목 사유 명",
        "Type": "String",
        "Required": "Y",
        "Length": 40,
        "Description": "특정 경우에만 데이터 출력"
      },
      "mrkt_warn_cls_name": {
        "한글명": "시장 경고 구분 명",
        "Type": "String",
        "Required": "Y",
        "Length": 40,
        "Description": "\"투자환기\" / \"투자경고\""
      },
      "stck_sdpr": {
        "한글명": "주식 기준가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "bstp_cls_code": {
        "한글명": "업종 구분 코드",
        "Type": "String",
        "Required": "Y",
        "Length": 4,
        "Description": ""
      },
      "stck_prdy_clpr": {
        "한글명": "주식 전일 종가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "insn_pbnt_yn": {
        "한글명": "불성실 공시 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "fcam_mod_cls_name": {
        "한글명": "액면가 변경 구분 명",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": "특정 경우에만 데이터 출력"
      },
      "stck_prpr": {
        "한글명": "주식 현재가",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "prdy_vrss": {
        "한글명": "전일 대비",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "prdy_vrss_sign": {
        "한글명": "전일 대비 부호",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "prdy_ctrt": {
        "한글명": "전일 대비율",
        "Type": "String",
        "Required": "Y",
        "Length": 82,
        "Description": ""
      },
      "acml_tr_pbmn": {
        "한글명": "누적 거래 대금",
        "Type": "String",
        "Required": "Y",
        "Length": 18,
        "Description": ""
      },
      "acml_vol": {
        "한글명": "누적 거래량",
        "Type": "String",
        "Required": "Y",
        "Length": 18,
        "Description": ""
      },
      "prdy_vrss_vol_rate": {
        "한글명": "전일 대비 거래량 비율",
        "Type": "String",
        "Required": "Y",
        "Length": 84,
        "Description": ""
      },
      "bstp_kor_isnm": {
        "한글명": "업종 한글 종목명",
        "Type": "String",
        "Required": "Y",
        "Length": 40,
        "Description": "※ 거래소 정보로 특정 종목은 업종구분이 없어 데이터 미회신"
      },
      "sltr_yn": {
        "한글명": "정리매매 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "trht_yn": {
        "한글명": "거래정지 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "oprc_rang_cont_yn": {
        "한글명": "시가 범위 연장 여부",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "vlnt_fin_cls_code": {
        "한글명": "임의 종료 구분 코드",
        "Type": "String",
        "Required": "Y",
        "Length": 1,
        "Description": ""
      },
      "stck_oprc": {
        "한글명": "주식 시가2",
        "Type": "String",
        "Required": "Y",
        "Length": 10,
        "Description": ""
      },
      "prdy_vol": {
        "한글명": "전일 거래량",
        "Type": "String",
        "Required": "Y",
        "Length": 18,
        "Description": ""
      }
    }
  }
}

In [ ]:
dfprcfield = pd.DataFrame(prcdict['Output']['fields']).T.reset_index()[['index', '한글명']]
dfprcfield.columns = ['code', 'name']

In [ ]:
dfprcfield

,code,name
0,rprs_mrkt_kor_name,대표 시장 한글 명
1,new_hgpr_lwpr_cls_code,신 고가 저가 구분 코드
2,mxpr_llam_cls_code,상하한가 구분 코드
3,crdt_able_yn,신용 가능 여부
4,stck_mxpr,주식 상한가
5,elw_pblc_yn,ELW 발행 여부
6,prdy_clpr_vrss_oprc_rate,전일 종가 대비 시가2 비율
7,crdt_rate,신용 비율
8,marg_rate,증거금 비율
9,lwpr_vrss_prpr,최저가 대비 현재가
